In [9]:
import sys
import os

sys.path += [
    '/home/ashwin/local/llama-agent-system',
    f'{os.path.expanduser("~/llama-agent-system")}'
]

# Uncomment your path in each list
MP1_CHECKPOINT_DIR = [
    # "/home/ashwin/local/checkpoints/llama3_7b_01_05_2024_torchx_launch_v5-qzn2md/",
    os.path.abspath(os.path.expanduser("~/models/mm_llama3_8b_final_non_te"))
][0]

MP1_TOKENIZER_PATH = [
    # "/home/ashwin/local/checkpoints/llama3_7b_01_05_2024_torchx_launch_v5-qzn2md/cl_toplang_128k",
    os.path.abspath(os.path.expanduser("~/models/mm_llama3_8b_final_non_te/cl_toplang_128k")),
][0]

MP8_CHECKPOINT_DIR = [
    os.path.abspath(os.path.expanduser("~/models/llama3_70b_full_rlhf3_14042024/"))
][0]

MP8_TOKENIZER_PATH = [
    os.path.abspath(os.path.expanduser("~/models/llama3_70b_full_rlhf3_14042024/cl_toplang_128k.model"))
][0]

os.chdir('/home/dalton/llama-agent-system/') # TODO: needed to get pwd for examples/ to work

In [10]:
from models.llama3 import (
    Attachment,
    Message,
    Image,
)

from toolchain.inference import GeneratorArgs, LlamaModelParallelGenerator
from agentic_system import (
    AgenticSystem,
    BraveSearchTool,
    CodeInterpreterTool,
    PhotogenTool,
    with_safety,
    WolframAlphaTool,
)

args = GeneratorArgs(
    ckpt_dir=MP1_CHECKPOINT_DIR,
    tokenizer_path=MP1_TOKENIZER_PATH,
    mock_generation=False,
    model_parallel_size=1,
    max_seq_len=2048,
    max_batch_size=4,
)

generator = LlamaModelParallelGenerator(args)
generator.start() # Loads model into memory

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1


INFO:root:Resizing tile embedding from 4x4 to 4x4
INFO:root:Resizing tile embedding from 4x4 to 4x4


Loaded in 13.34 seconds
NCCL version 2.20.5+cuda12.4
Finished model load YES READY


In [15]:
from PIL import Image as _PIL_Image

tools = [
        WolframAlphaTool(api_key="LKRWWW-J25AKURL7T"),
        BraveSearchTool(api_key="BSAo9hsA-Gr-d0O-QF65WesoYVqKoRO"),
        CodeInterpreterTool(),
        PhotogenTool(
            api_key="1078097093361062%7CHo16EWDKl2HmOHOEa426PBXVQB8",
            model="photorealism",
            dump_dir="/tmp/photogen_dump_" + os.environ["USER"],
        ),
    ]

agent = AgenticSystem(
        generator=generator,
        tools=tools,
        max_seq_len=2048,
        input_shields=[],
        output_shields=[],
    )

async def run(prompt):
     with open("examples/pasta.jpeg", "rb") as image_file:
        img = _PIL_Image.open(image_file).convert("RGB")

        result = await agent.run(
            dialog=[
                Message(
                    role="user",
                    content=[
                        Image(image=img),
                        prompt,
                    ],
                )
            ],
            temperature=0.4,
            top_p=0.9,
            max_gen_len=2048,
        )

        print(result['generation'].content)

In [16]:
await run("What is missing from this picture to make a nutritionally balanced meal?")

[debug] Got EOT generation(0) => 
To make this spaghetti di...
To make this spaghetti dish nutritionally balanced, the following components are missing:

1. Protein: Adding lean protein sources such as chicken, turkey, or tofu would provide essential amino acids and help balance the meal.
2. Vegetables: Incorporating a variety of colorful vegetables, such as bell peppers, broccoli, or spinach, would add fiber, vitamins, and minerals, as well as texture and flavor to the dish.
3. Healthy fats: Adding sources of healthy fats, such as olive oil, avocado, or nuts, would provide essential fatty acids and help balance the meal.
4. Whole grains: Using whole grain pasta or adding whole grain bread or crackers would provide additional fiber and nutrients.

By incorporating these missing components, the spaghetti dish can be made more nutritionally balanced and satisfying.


In [ ]:
await run("Can you recreate an image that has ?")

In [8]:
generator.stop() # Release the GPU memory after you're done